### Config stuff

In [1]:
import random
import pyspark
from pyspark.sql import SparkSession, functions
import ConnectionConfigKaloyan as cc
from pyspark.sql.functions import *
cc.setupEnvironment()
cc.listEnvironment()

ALLUSERSPROFILE: C:\ProgramData
APPDATA: C:\Users\kkiva\AppData\Roaming
COMMONPROGRAMFILES: C:\Program Files\Common Files
COMMONPROGRAMFILES(X86): C:\Program Files (x86)\Common Files
COMMONPROGRAMW6432: C:\Program Files\Common Files
COMPUTERNAME: DESKTOP-DJQSI3S
COMSPEC: C:\WINDOWS\system32\cmd.exe
DRIVERDATA: C:\Windows\System32\Drivers\DriverData
FPS_BROWSER_APP_PROFILE_STRING: Internet Explorer
FPS_BROWSER_USER_PROFILE_STRING: Default
HOMEDRIVE: C:
HOMEPATH: \Users\kkiva
IPY_INTERRUPT_EVENT: 1976
JPY_INTERRUPT_EVENT: 1976
JPY_PARENT_PID: 1388
JPY_SESSION_NAME: examples/05_2_DWH_dimUser_Initial.ipynb
LANG: en_US.UTF-8
LANGUAGE: 
LC_ALL: en_US.UTF-8
LOCALAPPDATA: C:\Users\kkiva\AppData\Local
LOGONSERVER: \\DESKTOP-DJQSI3S
NUMBER_OF_PROCESSORS: 8
ONEDRIVE: C:\Users\kkiva\OneDrive - KdG
ONEDRIVECOMMERCIAL: C:\Users\kkiva\OneDrive - KdG
OS: Windows_NT
PATH: C:\Users\kkiva\data4_project_group5\myvenv\Scripts;C:\Program Files\Microsoft\jdk-17.0.14.7-hotspot\bin;C:\Program Files (x86)\Commo

In [2]:
spark = cc.startLocalCluster("dimUserInitial",1)
spark.getActiveSession()


# Creating the operational database
In order to run this demo you have to create a tutorial_op database and run the PostgreSQL_SalesOperational.sql script.


# Initial load
We will create a slowly changing dimension type 2 called dimSalesRep based on a sourceTable in our operational database called dbo.salesrep. SCD2  tables demand extra care and  because we will store hirstorical values of the dimension with the help of ranges.
This notebook will create the table and fill it with the initial data. A second notebook will be used for increments of new and changed data.

This is an example of the expected output (salesRepSK is different
```
+----------+-------------+-------------+-----------+-------------------+-------------------+--------------------+-------+
|salesRepID|         name|       office| salesRepSK|          scd_start|            scd_end|                 md5|current|
+----------+-------------+-------------+-----------+-------------------+-------------------+--------------------+-------+
|a46add1...|      Z. Jane|     New York|          0|1990-01-01 00:00:00|2100-12-12 00:00:00|303db545462092a92...|   true|
|s1fedf1...|   P. Chapman|       Berlin|          1|1990-01-01 00:00:00|2100-12-12 00:00:00|14b094c31bf9e4149...|   true|
|d5e6f77...|     T. Crane|     New York|          2|1990-01-01 00:00:00|2100-12-12 00:00:00|6c062f95defda9dc3...|   true|
```




In [3]:

# Set up JDBC connection
cc.set_connectionProfile("default")

df_users = spark.read \
    .format("jdbc") \
    .option("driver", cc.get_Property("driver")) \
    .option("url", cc.create_jdbc()) \
    .option("dbtable", "velo_users") \
    .option("postgres", cc.get_Property("username")) \
    .option("strongage25", cc.get_Property("password")) \
    .option("partitionColumn", "userid") \
    .option("numPartitions", 1) \
    .option("lowerBound", 0) \
    .option("upperBound", 20) \
    .load()

# Create a temporary view
df_users.createOrReplaceTempView("dimUser")
spark.sql("select * from dimUser")

users_dim = spark.sql("select uuid() as userSK, *, to_timestamp('1999-01-01','yyyy-MM-dd') as scd_start, to_timestamp('2100-12-12','yyyy-MM-dd') as scd_end, md5(concat( street, number, zipcode, city, country_code)) as md5, True as current from dimUser")

users_dim.printSchema()


#delta files
#users_dim.write.format("delta").mode("overwrite").saveAsTable("dimUser")

#parquet files
users_dim.repartition(1).write.format("parquet").mode("overwrite").saveAsTable("dimUser_pq")

root
 |-- userSK: string (nullable = false)
 |-- userid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- street: string (nullable = true)
 |-- number: string (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- scd_start: timestamp (nullable = true)
 |-- scd_end: timestamp (nullable = true)
 |-- md5: string (nullable = true)
 |-- current: boolean (nullable = false)



## Delete the spark session

In [8]:
spark.stop()